In [12]:
import requests
import pandas as pd
companies = []
demo = 'd52136af660e914f9c80434a6738b7ff'
marketcap = str(1000000000)
url = (f'https://financialmodelingprep.com/api/v3/stock-screener?marketCapMoreThan={marketcap}&betaMoreThan=1&volumeMoreThan=10000&sector=Technology&exchange=NASDAQ&dividendMoreThan=0&limit=1000&apikey={demo}')
#get companies based on criteria defined about
screener = requests.get(url).json()


In [13]:
#add selected companies to a list
for item in screener:
 companies.append(item['symbol'])
 
#print(companies)
value_ratios ={}
#get the financial ratios
count = 0
for company in companies:
 try:
  if count <30:
   count = count + 1
   fin_ratios = requests.get(f'https://financialmodelingprep.com/api/v3/ratios/{company}?apikey={demo}').json()
   value_ratios[company] = {}
   value_ratios[company]['ROE'] = fin_ratios[0]['returnOnEquity']
   value_ratios[company]['ROA'] = fin_ratios[0]['returnOnAssets']
   value_ratios[company]['Debt_Ratio'] = fin_ratios[0]['debtRatio']
   value_ratios[company]['Interest_Coverage'] = fin_ratios[0]['interestCoverage']
   value_ratios[company]['Payout_Ratio'] = fin_ratios[0]['payoutRatio']
   value_ratios[company]['Dividend_Payout_Ratio'] = fin_ratios[0]['dividendPayoutRatio']
   value_ratios[company]['PB'] = fin_ratios[0]['priceToBookRatio']
   value_ratios[company]['PS'] = fin_ratios[0]['priceToSalesRatio']
   value_ratios[company]['PE'] = fin_ratios[0]['priceEarningsRatio']
   value_ratios[company]['Dividend_Yield'] = fin_ratios[0]['dividendYield']
   value_ratios[company]['Gross_Profit_Margin'] = fin_ratios[0]['grossProfitMargin']
   #more financials on growth:https://financialmodelingprep.com/api/v3/financial-growth/AAPL?apikey=demo
   growth_ratios = requests.get(f'https://financialmodelingprep.com/api/v3/financial-growth/{company}?apikey={demo}').json()
   value_ratios[company]['Revenue_Growth'] = growth_ratios[0]['revenueGrowth']
   value_ratios[company]['NetIncome_Growth'] = growth_ratios[0]['netIncomeGrowth']
   value_ratios[company]['EPS_Growth'] = growth_ratios[0]['epsgrowth']
   value_ratios[company]['RD_Growth'] = growth_ratios[0]['rdexpenseGrowth']
        
 except:
  pass


In [17]:
df = pd.DataFrame.from_dict(value_ratios,orient='index')
print(df.head(4))

           ROE       ROA  Debt_Ratio  Interest_Coverage  Payout_Ratio  \
AAPL  0.878664  0.177256    0.798267                NaN      0.245267   
NVDA  0.229105  0.161478    0.295178          57.115385      0.139485   
ASML  0.242694  0.142394    0.413278                NaN     -0.007333   
QCOM  0.855356  0.146036    0.829269           9.500000      0.554444   

      Dividend_Payout_Ratio         PB         PS         PE  Dividend_Yield  \
AAPL               0.245267  30.553901   7.272322  34.773150        0.007053   
NVDA               0.139485  13.252039  14.812958  57.842590        0.002411   
ASML              -0.007333   7.037145   6.864817  28.995971       -0.000253   
QCOM               0.554444  23.516185   6.073174  27.492854        0.020167   

      Gross_Profit_Margin  Revenue_Growth  NetIncome_Growth  EPS_Growth  \
AAPL             0.382332        0.055121          0.039000    0.113934   
NVDA             0.619894       -0.068112         -0.324801   -0.317153   
ASML    

In [16]:
#criteria ranking
ROE = 1.2
ROA = 1.1
Debt_Ratio = -1.1
Interest_Coverage = 1.05
Dividend_Payout_Ratio = 1.01
PB = -1.10
PS = -1.05
Revenue_Growth = 1.25
Net_Income_Growth = 1.10

In [5]:
#mean to enable comparison across ratios
ratios_mean = []
for item in df.columns:
 ratios_mean.append(df[item].mean())
#divide each value in dataframe by mean to normalize values
df = df / ratios_mean

In [18]:
df['ranking'] = df['NetIncome_Growth']*Net_Income_Growth + df['Revenue_Growth']*Revenue_Growth  + df['ROE']*ROE + df['ROA']*ROA + df['Debt_Ratio'] * Debt_Ratio + df['Interest_Coverage'] * Interest_Coverage  + df['PB']*PB + df['PS']*PS

In [19]:
print(df.sort_values(by=['ranking'],ascending=False))

           ROE       ROA  Debt_Ratio  Interest_Coverage  Payout_Ratio  \
ADSK -1.542056  0.034713    1.022511          -5.520599      0.000000   
ANSS  0.130682  0.093264    0.286328         150.991910      0.000000   
CTSH  0.167120  0.113676    0.319798          97.807692      0.245928   
SNPS  0.130386  0.083115    0.361628          46.788404      0.000000   
NVDA  0.229105  0.161478    0.295178          57.115385      0.139485   
GRMN  0.198704  0.154454    0.222693          27.409129      0.438079   
MRVL  0.182563  0.142312    0.220480           9.323516      0.100716   
TXN   0.563265  0.278444    0.505661          33.694118      0.599561   
CDNS  0.470294  0.294582    0.373621          25.437782      0.000000   
TER   0.315823  0.167731    0.468909          22.716440      0.131143   
ZBRA  0.295813  0.115474    0.609637           6.719101      0.000000   
NXPI  0.025739  0.012140    0.517636           0.786486      1.312757   
MCHP  0.102157  0.032744    0.679475           0.30